# 方法1 验证test acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
from transformers import ViTForImageClassification, ViTFeatureExtractor
from datasets import load_dataset
import os
from PIL import Image
import torch

In [ ]:
import torchvision.transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# 设定设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 检查点路径 - 假设你已经有了一个保存在"/content/drive/MyDrive/try_dataset/output/checkpoint-X"的检查点
checkpoint_path = "/content/drive/MyDrive/urbandata/Classification/transformer/output-14-test/checkpoint-340"  # 替换为具体的检查点编号

# 加载模型
model = ViTForImageClassification.from_pretrained(checkpoint_path).to(device)

# 测试数据集路径
test_data_dir = "/content/drive/MyDrive/urbandata/Classification/dataset/test"


In [ ]:
# 创建图像特征提取器
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

# 获取测试集中的所有类别（标签）
labels = sorted(os.listdir(test_data_dir))
predicted_labels=[]

# 初始化每个类别的样本数量和正确预测的数量
class_correct = {label: 0 for label in labels}
class_total = {label: 0 for label in labels}

# 遍历测试集并计算每个类别上的准确度
for label in labels:
    label_dir = os.path.join(test_data_dir, label)
    images = os.listdir(label_dir)
    for image_name in images:
        image_path = os.path.join(label_dir, image_name)

        # 进行预测
        img = Image.open(image_path)
        pixel_values = feature_extractor(images=img, return_tensors="pt")["pixel_values"].to(device)
        output = model(pixel_values)
        predicted_label = model.config.id2label[int(output.logits.softmax(dim=1).argmax())]

        predicted_labels.append(predicted_label)

        # 更新类别的样本数量和预测正确的数量
        class_total[label] += 1
        if predicted_label == label:
            class_correct[label] += 1

# 计算每个类别上的准确度
for label in labels:
    accuracy = class_correct[label] / class_total[label] if class_total[label] > 0 else 0
    print(f"Accuracy for {label}: {accuracy * 100:.2f}%")


In [ ]:
print(labels)
print(predicted_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# 计算 F1-score
f1 = f1_score(predicted_labels, labels, average='micro')
conf_matrix = confusion_matrix(predictions.label_ids, preds)
#predictions.label_ids真实 , preds预测

print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
cf_matrix=confusion_matrix(predicted_labels, labels)
sns.heatmap(cf_matrix, annot=True)

# 方法2 对图片分类

## Link to Google Drive and load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers evaluate datasets sentencepiece accelerate

import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 750.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00


## prediction

In [ ]:
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

In [ ]:
#加载训练好的模型
# the model name
model_name = "google/vit-base-patch16-224"
# load the image processor
image_processor = ViTImageProcessor.from_pretrained(model_name)

# loading the pre-trained model
checkpoint_path = "/content/drive/MyDrive/urbandata/Classification/transformer/output-14-test/checkpoint-340"
# 加载训练好的模型
model = ViTForImageClassification.from_pretrained(checkpoint_path).to(device)

In [ ]:
import urllib.parse as parse
import os

# a function to determine whether a string is a URL or not
def is_url(string):
    try:
        result = parse.urlparse(string)
        return all([result.scheme, result.netloc, result.path])
    except:
        return False

# a function to load an image
def load_image(image_path):
    if is_url(image_path):
        return Image.open(requests.get(image_path, stream=True).raw)
    elif os.path.exists(image_path):
        return Image.open(image_path)

In [ ]:
def get_prediction(model, url_or_path):
  # load the image
  img = load_image(url_or_path)
  # preprocessing the image
  pixel_values = image_processor(img, return_tensors="pt")["pixel_values"].to(device)
  # perform inference
  output = model(pixel_values)
  # get the label id and return the class name
  return model.config.id2label[int(output.logits.softmax(dim=1).argmax())]

In [ ]:
##Give it a try
get_prediction(model, "http://images.cocodataset.org/test-stuff2017/000000000128.jpg")

'3plant'

In [ ]:
# 文件夹路径 修改！！！！！！！！！
folder_path = '/content/drive/MyDrive/urbandata/Fliker/Tokyo/4'

# 创建一个空的预测结果列表
result_list = []

# 遍历文件夹中的图片文件
for idx, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)

        # 对图像进行预测
        try:
            img = Image.open(image_path)
            img = img.convert("RGB")

            # 使用之前定义的 get_prediction 函数进行预测
            predicted_class = get_prediction(model, image_path)

            # 将预测结果添加到结果列表
            result_list.append([filename, predicted_class])
        except Exception as e:
            print(f"Error occurred while processing {filename}: {e}")

In [ ]:
print(result_list)

[['_3249.jpg', '3plant'], ['_3250.jpg', '5car'], ['_3251.jpg', '3plant'], ['_3252.jpg', '3plant'], ['_3253.jpg', '3plant'], ['_3255.jpg', '3plant'], ['_3256.jpg', '3plant'], ['_3257.jpg', '3plant'], ['_3258.jpg', '3plant'], ['_3259.jpg', '3plant'], ['_3260.jpg', '6rest'], ['_3261.jpg', '6rest'], ['_3262.jpg', '3plant'], ['_3263.jpg', '3plant'], ['_3264.jpg', '1scrawl'], ['_3265.jpg', '4shop2'], ['_3266.jpg', '3plant'], ['_3267.jpg', '3plant'], ['_3268.jpg', '1scrawl'], ['_3269.jpg', '3plant'], ['_3270.jpg', '3plant'], ['_3271.jpg', '3plant'], ['_3272.jpg', '3plant'], ['_3273.jpg', '3plant'], ['_3274.jpg', '1scrawl'], ['_3275.jpg', '5bike'], ['_3276.jpg', '1scrawl'], ['_3277.jpg', '1scrawl'], ['_3278.jpg', '4shop1'], ['_3279.jpg', '6talk'], ['_3280.jpg', '3plant'], ['_3281.jpg', '1art_parade'], ['_3282.jpg', '1art_parade'], ['_3283.jpg', '3plant'], ['_3284.jpg', '1art_parade'], ['_3285.jpg', '6talk'], ['_3286.jpg', '1scrawl'], ['_3287.jpg', '1scrawl'], ['_3288.jpg', '3plant'], ['_3289.j

In [ ]:
import pandas as pd
csv_path='/content/drive/MyDrive/urbandata/csv_predicition/Tokyo/4'
# 将预测结果保存到 CSV 文件 修改！！！！！！！！！！
df = pd.DataFrame(result_list, columns=['Image', 'Predicted_Class'])
df.to_csv(csv_path+'.csv', index=False)
df.head(5)

In [ ]:
#修改label名称（14要做，6种的粗分类不需要）
import pandas as pd
# 读取文件
csv_path='/content/drive/MyDrive/urbandata/csv_predicition/Tokyo/4'
data = pd.read_csv(csv_path+'.csv')

# 原来的值：新的值
# 使用map属性
change= {'1art_parade':'1','1scrawl':'2','2political_parade':'3','3fieldwalk':'4','3plant':'5','4shop1':'6','4shop2':'7','5bike':'8','5car':'9','5skid':'10','5walkpeople':'11','6party':'12','6rest':'13','6talk':'14'}
data['Predicted_Class'] = data['Predicted_Class'].map(change)

#生成新的文件
data.to_csv(csv_path+'_new'+'.csv')
data = pd.read_csv(csv_path+'_new'+'.csv', nrows=5)
print(data)

   Unnamed: 0      Image  Predicted_Class
0           0  _3250.jpg                5
1           1  _3251.jpg                5
2           2  _3252.jpg                5
3           3  _3253.jpg                5
4           4  _3254.jpg                5
